In [124]:
# Install required packages.
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


In [125]:
data_dir = 'c:/Users/david/MT_data/extracted_patches/mutant_graphs_classification/'

In [ ]:
def visualize_points(pos, edge_index=None, index=None):
    fig = plt.figure(figsize=(4, 4))
    if edge_index is not None:
        for (src, dst) in edge_index.t().tolist():
             src = pos[src].tolist()
             dst = pos[dst].tolist()
             plt.plot([src[0], dst[0]], [src[1], dst[1]], linewidth=1, color='black')
    if index is None:
        plt.scatter(pos[:, 0], pos[:, 1], s=10.0, zorder=1000)
    else:
       mask = torch.zeros(pos.size(0), dtype=torch.bool)
       mask[index] = True
       plt.scatter(pos[~mask, 0], pos[~mask, 1], s=10.0, color='lightgray', zorder=1000)
       plt.scatter(pos[mask, 0], pos[mask, 1], s=10.0, zorder=1000)
    plt.axis('off')
    plt.show()

In [ ]:
visualize_points(data.pos, data.edge_index)

# Show an example Graph

In [126]:
path = os.path.join(data_dir, 'AAAH_GraphPatch.pkl')

In [127]:
from f_helper_functions import *

patch = load_object(path)
print(patch.edge_index.shape)
print(patch.edge_weight.shape)
print(patch.fitness)
print(patch.A.shape)
print(patch.mutant)
print()
print(patch)

(2, 6565)
(6565, 1)
0.0
(1073, 1073)
AAAH

            Number of Nodes: 1073
            Features: (1073, 16)
            Adjacency Matrix: (1073, 1073)
            Edge Weights (Geodesic Distances): (6565, 1)
            Edge Index: (2, 6565)
            Fitness: 0.0
            Coordinates of Points: (1073, 3)
            Mutant Name: AAAH


## Define customized Dataset

In [128]:
#from torch.utils.data import Dataset
from c_PatchDataset_classification import PatchDataset

batch_size = 8
dataset = PatchDataset(data_dir = data_dir)
len(dataset)

1500

In [129]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')


data = dataset[0]  # Get the first graph object.

print()
print(data)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of node features: {data.num_node_features}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Contains self-loops: {data.has_self_loops()}')


Dataset: PatchDataset(1500):
Number of graphs: 1500

Data(x=[1049, 16], edge_index=[2, 6435], edge_attr=[6435, 1], y=1, pos=[1049, 3])
Number of nodes: 1049
Number of node features: 16
Number of edges: 6435
Average node degree: 6.13
Contains self-loops: True


In [130]:
data.x

tensor([[0.1928, 0.6956, 0.2501,  ..., 0.6746, 0.7041, 0.7508],
        [0.2233, 0.7312, 0.2328,  ..., 0.7339, 0.7632, 0.7021],
        [0.2178, 0.7439, 0.2152,  ..., 0.7277, 0.7382, 0.7223],
        ...,
        [0.3455, 0.7100, 0.2421,  ..., 0.7356, 0.7057, 0.5601],
        [0.3573, 0.7389, 0.2096,  ..., 0.7989, 0.7327, 0.5888],
        [0.3467, 0.7444, 0.1991,  ..., 0.8134, 0.7271, 0.6105]])

In [131]:
len(dataset)

1500

In [132]:
n_train = int(len(dataset)*0.8)
n_test = len(dataset) - n_train 

trainset, testset = torch.utils.data.random_split(dataset, [n_train, n_test])

print(f'Number of training graphs: {len(trainset)}')
print(f'Number of test graphs: {len(testset)}')

Number of training graphs: 1200
Number of test graphs: 300


In [133]:
from torch_geometric.loader import DataLoader

trainloader = DataLoader(dataset = trainset, batch_size= batch_size, shuffle = True)
testloader = DataLoader(dataset = testset, batch_size= batch_size, shuffle = True)

## Training a Graph Neural Network (GNN)

Training a GNN for graph classification usually follows a simple recipe:

1. Embed each node by performing multiple rounds of message passing
2. Aggregate node embeddings into a unified graph embedding (**readout layer**)
3. Train a final classifier on the graph embedding

There exists multiple **readout layers** in literature, but the most common one is to simply take the average of node embeddings:

$$
\mathbf{x}_{\mathcal{G}} = \frac{1}{|\mathcal{V}|} \sum_{v \in \mathcal{V}} \mathcal{x}^{(L)}_v
$$

PyTorch Geometric provides this functionality via [`torch_geometric.nn.global_mean_pool`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.glob.global_mean_pool), which takes in the node embeddings of all nodes in the mini-batch and the assignment vector `batch` to compute a graph embedding of size `[batch_size, hidden_channels]` for each graph in the batch.

The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

In [134]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_max_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(16, 16, add_self_loops=False)
        self.conv2 = GCNConv(16, 12, add_self_loops=False)
        self.conv3 = GCNConv(12, 8, add_self_loops=False)
        self.conv4 = GCNConv(8, 5, add_self_loops=False)
        self.lin = Linear(5, 2)            

    def forward(self, x, edge_index, edge_weight, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_weight)

        # 2. Readout layer
        # Take the sum over all nodes in each graph = 16 values per graph
        x = global_max_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = self.lin(x)
        
        return x

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(16, 16)
  (conv2): GCNConv(16, 12)
  (conv3): GCNConv(12, 8)
  (conv4): GCNConv(8, 5)
  (lin): Linear(in_features=5, out_features=2, bias=True)
)


Here, we again make use of the [`GCNConv`](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv) with $\mathrm{ReLU}(x) = \max(x, 0)$ activation for obtaining localized node embeddings, before we apply our final classifier on top of a graph readout layer.

Let's train our network for a few epochs to see how well it performs on the training as well as test set:

In [135]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    for data in trainloader:  # Iterate in batches over the training dataset.
        print(data)
        out = model(data.x, data.edge_index, data.edge_weight, data.batch)  # Perform a single forward pass.
        print(out.shape)
        print(data.y)
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.


def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.edge_weight, data.batch)
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 10):
    loss = train()
    
    train_acc = test(trainloader)
    test_acc = test(testloader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

DataBatch(x=[8163, 16], edge_index=[2, 49909], edge_attr=[49909, 1], y=[8], pos=[8163, 3], batch=[8163], ptr=[9])
torch.Size([8, 5])
tensor([0, 1, 0, 0, 1, 1, 0, 0])
DataBatch(x=[8651, 16], edge_index=[2, 52842], edge_attr=[52842, 1], y=[8], pos=[8651, 3], batch=[8651], ptr=[9])
torch.Size([8, 5])
tensor([1, 1, 0, 0, 1, 1, 1, 0])
DataBatch(x=[8610, 16], edge_index=[2, 52650], edge_attr=[52650, 1], y=[8], pos=[8610, 3], batch=[8610], ptr=[9])
torch.Size([8, 5])
tensor([0, 1, 0, 0, 1, 0, 0, 1])
DataBatch(x=[8304, 16], edge_index=[2, 50684], edge_attr=[50684, 1], y=[8], pos=[8304, 3], batch=[8304], ptr=[9])
torch.Size([8, 5])
tensor([0, 1, 0, 1, 1, 1, 0, 0])
DataBatch(x=[8424, 16], edge_index=[2, 51535], edge_attr=[51535, 1], y=[8], pos=[8424, 3], batch=[8424], ptr=[9])
torch.Size([8, 5])
tensor([1, 0, 1, 1, 1, 0, 1, 0])
DataBatch(x=[8326, 16], edge_index=[2, 50934], edge_attr=[50934, 1], y=[8], pos=[8326, 3], batch=[8326], ptr=[9])
torch.Size([8, 5])
tensor([0, 0, 1, 0, 0, 0, 1, 1])
Data

KeyboardInterrupt: 